# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

In [ ]:
!pip install phik -q

In [ ]:
!pip install --upgrade scikit-learn -q

In [ ]:
!pip install shap -q

In [ ]:
!pip install lightgbm --upgrade -q

In [ ]:
!pip install category_encoders -q

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import phik
import math
import shap
import lightgbm as lgb
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
from phik import resources, report
from phik.report import plot_correlation_matrix
from scipy import stats as st
from scipy.stats import randint, mode
from math import sqrt
from math import factorial
from matplotlib import pyplot as plt
from scipy.stats import binom, norm
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso
from sklearn.svm import SVC, SVR
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier,  DecisionTreeRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from category_encoders import BinaryEncoder
from sklearn.metrics import (
    confusion_matrix, 
    precision_score, 
    recall_score, 
    accuracy_score, 
    r2_score, 
    mean_squared_error, 
    mean_absolute_error,
    roc_auc_score,
    make_scorer,
    f1_score
)
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
#from imblearn.over_sampling import SMOTE

In [ ]:
data_autos = pd.read_csv('/datasets/autos.csv', sep = ',', parse_dates=['DateCrawled', 'DateCreated', 'LastSeen'])

In [ ]:
def inf(df, r):
    df1=df.head(r)
    df2=df.info()
    return display(df1, df2)

In [ ]:
inf(data_autos, 10)

**Данные загружены, соответствуют описанию, типы данных в порядке. Имеются пропуски в данных.**

In [ ]:
data_autos.isna().sum()

**Обработаем пропуски.**

**Пропуски в наименованиях модели заменим на 'unknown'**

**Пропуски в `Repaired` также заменим на 'unknown'**

**Тип кузова заполним по наиболее распространенным значеним в сочетании марки и модели машины.**

**Тип коробки передач и тип топлива заполним аналогично типу кузова, по наиболее распространенным среди машин данной марки и модели.**

In [ ]:
data_autos['Model'] = data_autos['Model'].fillna('unknown')

In [ ]:
data_autos['Repaired'] = data_autos['Repaired'].fillna('unknown')

In [ ]:
def most_commons (col):
    most_com = data_autos.groupby(['Model', 'Brand'])[col].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
    data_autos[col] = data_autos[col].fillna(most_com)

In [ ]:
most_commons ('VehicleType')

In [ ]:
most_commons ('Gearbox')

In [ ]:
most_commons ('FuelType')

In [ ]:
data_autos.isna().sum()

**Проверим данные на дубликаты и удалим их, в случае наличия.**

In [ ]:
data_autos.duplicated().sum()

In [ ]:
data_autos = data_autos.drop_duplicates(keep='first')
data_autos.info()

**Предобработка закончена, проведём исследовательский анализ данных.**

In [ ]:
data_autos.columns

In [ ]:
inf(data_autos, 10)

In [ ]:
g1_hist = ['Price', 'VehicleType', 'RegistrationYear', 'Gearbox',
       'Power', 'Model', 'Kilometer', 'RegistrationMonth', 'FuelType', 'Brand',
       'Repaired', 'NumberOfPictures', 'PostalCode']
g2_hist = ['Цена (евро)', 'Тип автомобильного кузова', 'Год регистрации автомобиля', 'Тип коробки передач',
           'Мощность (л. с.)', 'Модель автомобиля', 'Пробег (км)', 'Месяц регистрации автомобиля',
           'Тип топлива', 'Марка автомобиля', 'Была ли машина в ремонте', 'Количество фотографий автомобиля',
           'Почтовый индекс владельца анкеты']
bins_hist = [30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30]

In [ ]:
def hist_1(g1_hist, g2_hist, bins_hist, data_autos):
    for n in range(len(g1_hist)):
        discr_sign = ['RegistrationMonth', 'NumberOfPictures', 'PostalCode']

        if pd.api.types.is_numeric_dtype(data_autos[g1_hist[n]]) and (g1_hist[n] not in discr_sign):
            fig, ax = plt.subplots(nrows=2, ncols=1, gridspec_kw={'height_ratios': [3, 1]}, figsize=(8,6))
            ax[0].hist(data_autos[g1_hist[n]], bins=bins_hist[n], alpha=0.5, edgecolor='red')
            ax[0].set_title(f'Гистограмма и "ящик с усами" для признака \n{g2_hist[n]}', fontsize=14)
            ax[0].set_ylabel('Количество', fontsize=12)
            ax[0].grid(True)
            
            ax[1].boxplot(data_autos[g1_hist[n]], vert=False, widths=0.7, positions=[1], patch_artist=True, boxprops=dict(facecolor='red', color='red'))
            ax[1].set_xlabel(f'{g2_hist[n]}', fontsize=12)
            ax[1].set_yticks([1])
            
        elif pd.api.types.is_numeric_dtype(data_autos[g1_hist[n]]) and (g1_hist[n] in discr_sign):
            fig, ax = plt.subplots(figsize=(8,6))
            sns.countplot(x=data_autos[g1_hist[n]], ax=ax, color='red', alpha=0.5)
            ax.set_title(f'Диаграмма для признака \n{g2_hist[n]}', fontsize=14)
            ax.set_ylabel('Количество', fontsize=12)
            ax.set_xlabel(f'{g2_hist[n]}', fontsize=12)
            ax.grid(True)
                  

        else:

            fig, ax = plt.subplots(figsize=(12,6))
            value_counts = data_autos[g1_hist[n]].value_counts(normalize=True)
            wedges, texts = ax.pie(value_counts, startangle=90)
            ax.set_title(f'{g2_hist[n]}', fontsize=12)
            legend_labels = [f'{label}: {percent:.1f}%' for label, percent in zip(value_counts.index, value_counts * 100)]
            ax.legend(wedges, legend_labels, title="Категории", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
            
        plt.tight_layout()
        plt.show()

        display(data_autos[g1_hist[n]].describe(include='all'))
        print(80*'*')

In [ ]:
hist_1(g1_hist, g2_hist, bins_hist, data_autos)

In [ ]:
g1_hist_d=['DateCrawled', 'DateCreated', 'LastSeen']
g2_hist_d = ['Дата скачивания анкеты из базы', 'Дата создания анкеты', 'Дата последней активности пользователя']
bins_hist_d = [60, 60, 60]

In [ ]:
for n in range(len(g1_hist_d)):
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 6))
    ax.hist(data_autos[g1_hist_d[n]], bins=bins_hist_d[n], alpha=0.5, edgecolor='red')
    ax.set_title(f'Гистограмма и "ящик с усами" для признака \n{g2_hist_d[n]}', fontsize=14)
    ax.set_ylabel('Количество', fontsize=12)
    ax.grid(True)

    plt.show()

In [ ]:
data_autos['DateCreated'].max()

**По диаграммам можем отметить следующее:**
- Гистограмма по цене имеет вид распределения Пуассона с пиком около 0. Большая доля цен сосредоточена в нижнем диапазоне, однако разброс цен очень большой, и вследствие наличия высоких значений наблюдаем, что среднее и медианные значения довольно сильно различаются и стандартное отклонение довольно высокое. Ящик с усами показывает выбросы, однако цены правдоподобны, поэтому оставим эти значения.
- По круговой диаграмме типов автомобильного кузова видим наибольшую распространенность кузова типа sedan.
- По гистограмме года регистрации автомобиля видим наличие выбросов, неправдоподобных значений. Удалим строки с годом менее 1950 (автомобили более раннего года если и встречаются, то штучно и в качестве раритета, на который стандартные подходы к прогнозу цен не будут работать) и с годом более 2016, так как выгрузка ограничена этим годом.
- Коробка передач превалирует ручная (manual)
- По мощности видим выбросы, аномальные значения. Оставим значения до 1000 л.с., остальные удалим.
- По круговой диаграмме видим очень большой перечень моделей. Наибольшую долю занимает golf.
- Гистограмма по пробегу имеет вид распределения Пуассона с пиком около 150000 км, именно в этих значениях сосредоточено наибольшее количество объявлений. Среднее и медианные значения различаются не очень сильно, стандартное отклонение умеренное. Ящик с усами показывает выбросы, однако значения пробега правдоподобны, поэтому оставим их.
- По месяцам регистрации распределение довольно равномерное, при этом видим большое количество значений 0, что вероятно связано либо с ошибкой, либо просто с отсутствием данных.
- По типу топлива видим, что наибольшую долю занимает бензин (petrol).
- По маркам автомобиля видим лидерство за Volkswagen (21,7%).
- 69,7% машин заявляются, как не бывщие в ремонте. При этом по большой доле (20%) данные отсутствуют.
- Абсолютно по всем объявлениям отсутствуют фотографии автомобиля.
- Даты скачивания анкеты распределены равномерно
- Даты создания анкеты сосредоточены в диапазоне март-апрель 2016 года, а также в малом количестве в более ранние периоды, до 2014 года
- Даты последней активности пользователя сосредоточены в диапазоне 6-7 апреля 2016 года, а также в малом количестве в более ранние даты, до 5 марта 2016 года

**Также можем отметить неинформативный признак `NumberOfPictures` (количество фотографий автомобиля), его можно удалить.**

In [ ]:
data_autos = data_autos.query('1950<=RegistrationYear<=2016 and Power<1000')

In [ ]:
data_autos = data_autos.drop('NumberOfPictures', axis=1)

In [ ]:
data_autos.head()

In [ ]:
g1_hist = ['RegistrationYear', 'Power']
g2_hist = ['Год регистрации автомобиля', 'Мощность (л. с.)']
bins_hist = [76, 30]

In [ ]:
hist_1(g1_hist, g2_hist, bins_hist, data_autos)

**После удаления аномалий видим:**
- Гистограмма по году регистрации имеет вид распределения Пуассона с пиком около 2000 года. Среднее и медианные значения близки, стандартное отклонение небольшое. 
- Гистограмма по мощности имеет вид распределения Пуассона с пиком около 100 л.с. Среднее и медианные значения близки, стандартное отклонение довольно большое. 

In [ ]:
data_autos.info()

**После удалений аномалий, потеряно менее 5% данных, что является для нас допустимым.**

**Судя по почтовому индексам данные относятся к Германии. В почтовом индексе Германии код региона заложен в первых 2 цифрах, он нам более ценен, чем конкретное почтовое отделение. Извлечем их в отдельный столбце и удалим почтовый индекс.**

In [ ]:
data_autos['Region'] = data_autos['PostalCode'].astype(str).str[:2].astype(int)
data_autos = data_autos.drop('PostalCode', axis=1)

**Построим матрицу корреляций Phik.**

In [ ]:
phik_cols = ['Price', 'VehicleType', 'RegistrationYear', 'Gearbox',
       'Power', 'Model', 'Kilometer', 'RegistrationMonth', 'FuelType', 'Brand',
       'Repaired', 'Region']
phik_corr_matrix = (data_autos[phik_cols]
                    .phik_matrix(interval_cols= ['Price', 'RegistrationYear', 
       'Power', 'Kilometer'])
)
plt.figure(figsize=(15, 10))
sns.heatmap(phik_corr_matrix, cmap='viridis', annot=True, fmt=".2f")
plt.title('Матрица корреляций Phik')
plt.show()

**По матрице корреляций видим сильную зависимость целевого признака (Price) от года регистрации автомобиля, модели и мощности, а также умеренную зависимость от типа автомобильного кузова, типа коробки передач, пробега, бренда, наличия ремонта и индекса пользователя.**

**Также наблюдаем очень высокую корреляцию между моделью и типом кузова (0.91), а также маркой автомобиля (1.0), что говорит о наличии мультиколлинеарности. Объединим колонки с брендом, моделью и типом кузова в одну.**

In [ ]:
data_autos['Model']=data_autos['Brand']+ ' ' + data_autos['Model']+ ' ' + data_autos['VehicleType']
data_autos = data_autos.drop(['Brand', 'VehicleType'], axis=1)

In [ ]:
data_autos.head()

In [ ]:
phik_cols = ['Price', 'RegistrationYear', 'Gearbox',
       'Power', 'Model', 'Kilometer', 'RegistrationMonth', 'FuelType', 
       'Repaired', 'Region']
phik_corr_matrix = (data_autos[phik_cols]
                    .phik_matrix(interval_cols= ['Price', 'RegistrationYear', 
       'Power', 'Kilometer'])
)
plt.figure(figsize=(15, 10))
sns.heatmap(phik_corr_matrix, cmap='viridis', annot=True, fmt=".2f")
plt.title('Матрица корреляций Phik')
plt.show()

In [ ]:
data_autos.columns

In [ ]:
data_autos = data_autos.drop(['DateCrawled', 'DateCreated', 'LastSeen', 'RegistrationMonth'], axis=1)

## Обучение моделей

In [ ]:
RANDOM_STATE = 42
TEST_SIZE = 0.25

#label_encoder = LabelEncoder()
#data_autos['Price'] = label_encoder.fit_transform(data_autos['Price'])

X_train, X_test, y_train, y_test = train_test_split(
    data_autos.drop(columns=['Price'], axis=1),
    data_autos['Price'],
    test_size=TEST_SIZE, 
    random_state=RANDOM_STATE
)

be_columns = ['Gearbox', 'Model', 'FuelType', 'Repaired']
num_columns = ['RegistrationYear', 'Power', 'Kilometer', 'Region']

be_pipe = Pipeline(
    [('simpleImputer_before_ord', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
     ('binary_encoder', BinaryEncoder()),
     ('simpleImputer_after_ord', SimpleImputer(missing_values=np.nan, strategy='most_frequent'))
    ])


data_preprocessor = ColumnTransformer([
    ('binary_encoder', be_pipe, be_columns),
    ('num', MinMaxScaler(), num_columns)
], remainder='passthrough')

**P.S. Не смог сдружить модель линейной регрессии с OHE-кодировкой, видимо сильно нагружает её большое количество моделей, из-за чего падает Юпитер. Поэтому оставляю для линейной регрессии Ordinal-кодирование.**

In [ ]:
%%time
pipe_final = Pipeline([
    ('preprocessor', data_preprocessor),
    ('models',LinearRegression())
])

param_grid = [
    {
        'models': [LinearRegression()],
        'preprocessor__num': ['passthrough']
    }
]

rmse_scorer = make_scorer(lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)), greater_is_better=False)

randomized_search_linear = RandomizedSearchCV(
    pipe_final, 
    param_grid, 
    cv=5,
    scoring=rmse_scorer,
    random_state=RANDOM_STATE,
    n_jobs=-1
)
randomized_search_linear.fit(X_train, y_train)

print('Лучшая модель и её параметры:\n\n', randomized_search_linear.best_estimator_)
print('Метрика лучшей модели на тренировочной выборке:', randomized_search_linear.best_score_)

In [ ]:
def model_metrics(n):
    cv_results = n.cv_results_
    mean_fit_time = cv_results['mean_fit_time'].mean()
    mean_score_time = cv_results['mean_score_time'].mean()
    #mean_test_rmse = cv_results['mean_test_score'].mean()
    print(f'Среднее время обучения: {mean_fit_time}')
    print(f'Среднее время предсказания: {mean_score_time}')
    print(f'Средняя метрика RMSE на тренировочной выборке: {n.best_score_}')

In [ ]:
model_metrics(randomized_search_linear)

In [ ]:
%%time
pipe_final = Pipeline([
    ('preprocessor', data_preprocessor),
    ('models', DecisionTreeRegressor(random_state=RANDOM_STATE))
])

param_grid = [
    {
        'models': [DecisionTreeRegressor(random_state=RANDOM_STATE)],
        'models__max_depth': range(2, 30),
        'models__max_features': range(1, 8),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']
    }
]

rmse_scorer = make_scorer(lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)), greater_is_better=False)

randomized_search_tree = RandomizedSearchCV(
    pipe_final, 
    param_grid, 
    cv=5,
    scoring=rmse_scorer,
    random_state=RANDOM_STATE,
    n_jobs=-1
)
randomized_search_tree.fit(X_train, y_train)

print('Лучшая модель и её параметры:\n\n', randomized_search_tree.best_estimator_)
print('Метрика лучшей модели на тренировочной выборке:', randomized_search_tree.best_score_)

In [ ]:
model_metrics(randomized_search_tree)

In [ ]:
%%time
pipe_final = Pipeline([
    ('preprocessor', data_preprocessor),
    ('models', lgb.LGBMRegressor(random_state=42))
])

param_grid = [
    
    {
        'models': [lgb.LGBMRegressor(random_state=42)],
        'models__n_estimators': [50, 100, 200],
        'models__learning_rate': [0.01, 0.1, 0.3],
        'models__num_leaves': [20, 70, 150],
        'models__max_depth': [5, 10],
        'preprocessor__num': [MinMaxScaler()]
    }
]

rmse_scorer = make_scorer(lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)), greater_is_better=False)

randomized_search_lgb = RandomizedSearchCV(
    pipe_final, 
    param_grid, 
    cv=5,
    scoring=rmse_scorer,
    random_state=RANDOM_STATE,
    n_jobs=-1
)
randomized_search_lgb.fit(X_train, y_train)

print('Лучшая модель и её параметры:\n\n', randomized_search_lgb.best_estimator_)
print('Метрика лучшей модели на тренировочной выборке:', randomized_search_lgb.best_score_)

In [ ]:
model_metrics(randomized_search_lgb)

**Обучение моделей завершено, переходим к анализу.**

## Анализ моделей

**По итогу обучения моделей, получили следующие результаты:**

In [ ]:
print(f'Показатели LinearRegression():')
model_metrics(randomized_search_linear)

In [ ]:
print(f'Показатели DecisionTreeRegressor():')
model_metrics(randomized_search_tree)

In [ ]:
print(f'Показатели LGBMRegressor():')
model_metrics(randomized_search_lgb)

**Критерии, которые важны заказчику:**
- качество предсказания;
- время обучения модели;
- время предсказания модели.

**Значение метрики RMSE должно быть меньше 2500.**

**По данным критериям модель линейной регрессии нам не подходит. Модель градиентного бустинга имеет лучшие результаты по качеству предсказаний, но она более медленная по сравнению с деревом решений в 3-4 раза. Так как в требованиях заказчика указано, что значение метрики RMSE должно быть меньше 2500 и нет условий, что ещё меньшие значения повышают ценность модели, то принимаем это условие за достаточное (заказчик не испытывает необходимости в более высоком качестве модели) и рекомендуем заказчику более быструю модель из двух удовлетворяющих по значениям метрики RMSE, а именно DecisionTreeRegressor(), которая будет с большей скоростью обрабатывать большие объёмы данных с удовлетворяющим заказчика качеством предсказаний.**

**Проводим тестирование лучшей модели (DecisionTreeRegressor()).**

In [ ]:
%%time
y_test_pred = randomized_search_tree.predict(X_test)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
print(f'Метрика RMSE на тестовой выборке: {rmse}')
print(f'Параметры лучшей модели: {randomized_search_tree.best_params_}')

**На тестовых данных выбранная нами модель показала хорошие результаты:**
- время предсказания 0.2 с
- RMSE: 2343

**Рекомендация применения модели DecisionTreeRegressor() остаётся неизменной.**

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнена загрузка и подготовка данных
- [x]  Выполнено обучение моделей
- [x]  Есть анализ скорости работы и качества моделей